In [9]:
import os
os.chdir("/Users/r.s/Desktop/Gen AI Projects/krish naik/kidney disease classification/kidney_disease_classfication_using_deeplearning")

In [10]:
%pwd

'/Users/r.s/Desktop/Gen AI Projects/krish naik/kidney disease classification/kidney_disease_classfication_using_deeplearning'

In [18]:
from dataclasses import dataclass
from pathlib import Path
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier.constants import *

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir:Path
    base_model_path: Path
    updated_base_model_path: Path
    IMAGE_SIZE: list # as per VGG 16 model
    INCLUDE_TOP: bool
    CLASSES: int
    WEIGHTS: str
    LEARNING_RATE: float
    
    
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
        
    def get_base_model_config(self)->PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])
        
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            IMAGE_SIZE=self.params.IMAGE_SIZE,
            LEARNING_RATE=self.params.LEARNING_RATE,
            INCLUDE_TOP=self.params.INCLUDE_TOP,
            CLASSES=self.params.CLASSES,
            WEIGHTS=self.params.WEIGHTS,
            
        )
        
        return prepare_base_model_config
        
        


In [26]:
import tensorflow as tf

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        
    def get_base_model(self):
        self.model = tf.keras.applications.VGG16(
            include_top=self.config.INCLUDE_TOP,
            input_shape=self.config.IMAGE_SIZE
        )
        
        self.save_model(path=self.config.base_model_path, model=self.model)
        
    @staticmethod
    def prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till>0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(units=classes,
                                             activation="softmax")(flatten_in)
        
        full_model = tf.keras.models.Model(
            inputs = model.input,
            outputs = prediction)
        
        full_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
                                                             loss=tf.keras.losses.CategoricalCrossentropy(),
                                                             metrics=["accuracy"]
                                                             )
        full_model.summary()
        
        return full_model
        
    def update_base_model(self):
        self.full_model = self.prepare_full_model(model=self.model,
                                                  classes=self.config.CLASSES,
                                                  freeze_all=True,
                                                  freeze_till=None,
                                                  learning_rate=self.config.LEARNING_RATE)
        self.save_model(path=self.config.updated_base_model_path,
                        model=self.full_model)
          
                
        
    @staticmethod
    def save_model(path:Path, model: tf.keras.Model):
        model.save(path)
        
        
        
    
        

In [27]:
#pipeline

try:
    configuration = ConfigurationManager()
    base_model_config = configuration.get_base_model_config()
    base_model = PrepareBaseModel(config=base_model_config)
    base_model.get_base_model()
    base_model.update_base_model()
except Exception as e:
    raise e
    
    
    

[2024-05-27 19:19:20,910]: INFO: common: yaml file: config/config.yaml loaded successfully:
[2024-05-27 19:19:20,915]: INFO: common: yaml file: params.yaml loaded successfully:
[2024-05-27 19:19:20,917]: INFO: common: created directory at: artifacts:
[2024-05-27 19:19:20,918]: INFO: common: created directory at: artifacts/prepare_base_model:


[2024-05-27 19:19:21,471]: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.:
Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                       